In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate,ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1 
)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b},", 
)

prompt = template.format(country_a="Mexico",country_b="Thailand")

chat.predict(prompt)


In [ ]:
from langchain.schema import HumanMessage,AIMessage,SystemMessage


messages = [
    SystemMessage(content="You are a geographic expert. and you only reply in {language}.") ,
    AIMessage(content="Ciao, mi chiamo {name}"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, what is your name?"),
]

chat.predict_messages(messages)


In [ ]:
template = ChatPromptTemplate.from_messages(
    [
    ("system","You are a geographic expert. and you only reply in {language}."),
    ("ai","Ciao, mi chiamo {name}"),
    ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a = "Mexico",
    country_b = "Thailand",
)

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip,items))

p = CommaOutputParser()
p.parse("Hello,how, are, you")


In [ ]:
template = ChatPromptTemplate.from_messages(
    [
       ( "system"," You are a comma separated listing generating machine. Everything you are asked will be answered with a list of max {max_items} in lowercase. Do not reply with anlything else" ) ,
       ( "human","{question}" ) ,
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)


In [ ]:


chain = template | chat | CommaOutputParser()

chain.invoke({
     "max_items":5,
    "question":"What are the Pokemons?"
})



In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat 




In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human","{recipe}"),
])

veg_chain = veg_chef_prompt | chat 

final_chain = {"recipe":chef_chain} | veg_chain 

final_chain.invoke({
    "cuisine":"Indian",
})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate,FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


# prompt template을 디스크에 저장 후 load 할 수 있는 점이 좋다 
#t = PromptTemplate.from_template("What is the capital of {country} ?")
#t.format(country="France")

#FewShotPromptTemplate : LLM에게  Prompt 예제를 주는 거다 . 
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

#chat.predict("What do you know about France?")

example_template = PromptTemplate.from_template("Human:{question}\n AI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_template,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

#prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country":"Germany"})

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


#FewShotPromptTemplate : LLM에게  Prompt 예제를 주는 거다 . 
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai","{answer}")
    ]
) 
    

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a geographic expert. "),
        few_shot_prompt,
        ("human","What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country":"Turkey"})


        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira
        

AIMessageChunk(content='\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebabs and Baklava\n        Currency: Turkish Lira\n        ')

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

#FewShotPromptTemplate : LLM에게  Prompt 예제를 주는 거다 . 
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

    
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector=example_selector,
    suffix="What do you know about {country}?",
    input_variables=["country"]
)


prompt.format(country="Brazil")  #example 없이 작동 



'Human:France\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman:Italy\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nWhat do you know about Brazil?'

In [41]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector


 
#FewShotPromptTemplate : LLM에게  Prompt 예제를 주는 거다 . 
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self,examples):
       self.examples = examples
    
    def add_example(self, example):
       self.examples.append(example)

    def select_examples(self, input_variables):
      from random import choice

      return [choice(self.examples)]
    

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

    
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector=example_selector,
    suffix="What do you know about {country}?",
    input_variables=["country"]
)


prompt.format(country="Brazil")  #example 없이 작동 




'Human:France\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nWhat do you know about Brazil?'

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt    

#prompt = load_prompt("./prompt.json")   #Serialization
prompt = load_prompt("./prompt.yaml")   #Serialization

#FewShotPromptTemplate : LLM에게  Prompt 예제를 주는 거다 . 
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")



'What do you know about Germany?'

In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

#prompt = load_prompt("./prompt.json")   #Serialization
prompt = load_prompt("./prompt.yaml")   #Serialization

#FewShotPromptTemplate : LLM에게  Prompt 예제를 주는 거다 . 
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, it be a meal fit for a pirate like meself! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, it be a meal fit for a pirate like meself! Arrr!")

In [52]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache,set_debug
from langchain.cache import InMemoryCache,SQLiteCache


#set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))   #DB 에도 캐쉬 가능 
set_debug(True)   # LLM 이 내부에서 어떻게 진행되는지 로그를 남겨줌 

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make Italian pasta ? ")  #3.9 sec



[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta ?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [1.92s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [50]:
chat.predict("How do you make Italian pasta ? ")  #0.0 sec

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [54]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.callbacks import get_openai_callback


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

with get_openai_callback() as usage:
    a = chat.predict("What is the receip for Soju? ")  
    b = chat.predict("What is the receip for bread? ")  
    print(a,b,"\n")
    print(usage)



[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: What is the receip for Soju?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Ingredients:\n- 1 cup of rice\n- 1 cup of barley\n- 1 cup of water\n- 1 tablespoon of yeast\n- 1 tablespoon of sugar\n\nInstructions:\n1. Rinse the rice and barley in cold water until the water runs clear.\n2. In a large pot, combine the rice, barley, and water. Bring to a boil over high heat, then reduce heat to low and simmer for 30 minutes.\n3. Remove from heat and let cool to room temperature.\n4. In a small bowl, mix the yeast and sugar with a little warm water until dissolved. Add this mixture to the cooled rice and barley mixture.\n5. Cover the pot with a clean cloth and let it sit at room temperature for 3-4 days, stirring occasionally.\n6. After 3-4 days, strain the mixture through a cheesecloth or fine mesh strainer to remove any solids.\n7. 

In [60]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm


chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo"
)

chat.save("model.json")

# chat = load_llm("model.json")
# chat

